In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import requests
from bs4 import BeautifulSoup
import sqlite3

from name_errors import name_errors_dict

## ETL

The first step to answering this problem and building a model to predict contract value is to get the data we need. This means that I need data about the players, their stats for different seasons, and the free agents contracts that were signed. To do this, I will perform web scraping on the following webpages to get the needed data.

Player Data: https://www.basketball-reference.com/teams/BOS/2021.html \*Note: Taking data from 'Roster' Table. One webpage/table for each team and year.
<br>Stats Data: https://www.basketball-reference.com/leagues/NBA_2021_per_game.html \*Note: Taking data from 'Player Per Game' Table. One webpage/table for each year.
<br>Free Agent Contract Data: https://www.spotrac.com/nba/free-agents/2021/ \*Note: Taking data from main table. One webpage/table for each year.

After the data is scraped, I will add it to a sqlite database. Running this notebook will perform all the necessary steps for webscraping and adding the data to a database.

### Web Scraping

The first step in the ETL process is scrape three webpages to get player info data, statistics data, and free agent contract data.

To start, I will create an array with the years and teams that I am interested in.

In [2]:
YEARS = ['2019', '2020', '2021']

TEAMS = ['BOS', 'BRK', 'NYK', 'PHI', 'TOR',
         'CLE', 'IND', 'CHI', 'MIL', 'DET',
         'MIA', 'CHO', 'ORL', 'ATL', 'WAS',
         'NOP', 'DAL', 'SAS', 'MEM', 'HOU',
         'DEN', 'MIN', 'UTA', 'OKC', 'POR',
         'LAL', 'LAC', 'GSW', 'SAC', 'PHO']

The following function will get the player data for each team for every year. It will also create the player id dictionary that will be used for the statistics data and free agent contract data.

Data URL: https://www.basketball-reference.com/teams/BOS/2021.html
<br>*Note: Taking data from 'Roster' Table. One webpage/table for each team and year.

In [3]:
def get_player_data(years, teams):
    #this function will access the player information from each team in teams for each year in years
    
    player_id_dict = {}
    id_count = 1
    data = []

    for year in years:
        for team in teams:
            #this section gets the statistics table from Basketball-Reference.com
            web = requests.get('https://www.basketball-reference.com/teams/'+team+'/'+year+'.html').text
            soup = BeautifulSoup(web, 'lxml')
            table = soup.find('tbody').find_all('tr')
            
            #each row corresponds to one player's information
            for row in table:
                temp = row.find_all('td')
                temp = [x.text for x in temp]
                #if a player is already in player_id_dict (previous season or other team), we don't add them again
                if temp[0] in player_id_dict:
                    continue
                player_id_dict[temp[0]] = id_count #generate id for each player
                temp[2] = temp[2].split('-')
                if temp[6] == 'R': #change rookie to 0 years in league
                    temp[6] = '0'
                if temp[7] == '': #handles players who didn't go to college
                    temp[7] = 'None'
                data.append((id_count, temp[0], temp[1], (int(temp[2][0])*12 + int(temp[2][1])), int(temp[3]),
                             temp[4], temp[5].upper(), (int(year) - int(temp[6])), temp[7]))
                id_count+=1
                    
    return player_id_dict, data

In [4]:
player_id_dict, player_data = get_player_data(YEARS, TEAMS)

This next cell will get the length of the player_data array and an example tuple from this array.

In [5]:
print(len(player_data))
print(player_data[0])

756
(1, 'Aron Baynes', 'C', 82, 260, 'December 9, 1986', 'NZ', 2013, 'Washington State')


The following function will get the statistics data for each year.

Data URL: https://www.basketball-reference.com/leagues/NBA_2021_per_game.html
<br>*Note: Taking data from 'Player Per Game' Table. One webpage/table for each year.

In [6]:
def get_stats_data(years, player_id_dict):
    #this function will access the individual stats for the entire NBA for each year in years
    
    data = [['empty', 'empty', 'empty']]

    for year in years:
        #this section gets the statistics table from Basketball-Reference.com
        web = requests.get('https://www.basketball-reference.com/leagues/NBA_'+year+'_per_game.html').text
        soup = BeautifulSoup(web, 'lxml')
        table = soup.find('tbody').find_all('tr')
        
        #each row corresponds to one player's stats
        for row in table:
            temp = row.find_all('td')
            temp = [x.text for x in temp]
            
            #this if block ignores rows not associated with a player and handles the situation where a player played
            #for more than one team in a season.
            if temp == [] or temp[0]==data[-1][2]: 
                continue
            temp.pop(1) #get rid of the position
            for i in [1, 3, 4]: #set these statistics to integers
                temp[i] = int(temp[i])
            for i in range(5, len(temp)): #set these statistics to floats
                try:
                    temp[i] = float(temp[i])
                except:
                    temp[i] = 0
                #try except block handles case where player has undefined percentage stats, sets to 0
            
            
            #gets player_id from player_id_dict, except block handles case where players name is different among pages
            try:
                p_id = player_id_dict[temp[0]]
            except KeyError:
                if temp[0] in name_errors_dict:
                    p_id = player_id_dict[name_errors_dict[temp[0]]]
                if ' '.join(temp[0].split(' ')[:2]) in player_id_dict:
                    p_id = player_id_dict[' '.join(temp[0].split(' ')[:2])]
                else:
                    continue
                
            temp.insert(0, int(year))
            temp.insert(0, p_id)
            
            
            data.append(tuple(temp))
    
    data.pop(0)
    
    return data

In [7]:
stats_data = get_stats_data(YEARS, player_id_dict)

This next cell will get the length of the stats_data array and an example tuple from this array.

In [8]:
print(len(stats_data))
print(stats_data[0])

1594
(425, 2019, 'Álex Abrines', 25, 'OKC', 31, 2, 19.0, 1.8, 5.1, 0.357, 1.3, 4.1, 0.323, 0.5, 1.0, 0.5, 0.487, 0.4, 0.4, 0.923, 0.2, 1.4, 1.5, 0.6, 0.5, 0.2, 0.5, 1.7, 5.3)


The following function will get the free agent contract data for each year.

Data URL: https://www.spotrac.com/nba/free-agents/2021/
<br>*Note: Taking data from main table. One webpage/table for each year.

In [9]:
def get_free_agent_data(years, player_id_dict):
    #this function will access the contract information for all the free agents for each year in years
    data = []
    
    for year in years:
        #this section gets the statistics table from Spotrac.com
        web = requests.get('https://www.spotrac.com/nba/free-agents/'+year+'/').text
        soup = BeautifulSoup(web, 'lxml')
        table = soup.find('tbody').find_all('tr')
        
        #each row corresponds to one contract
        for row in table:
            temp = row.find_all('td')
            temp = [x.text.strip() for x in temp]
            if temp[7]=='0-': #disregard free agents who didn't sign contracts
                continue
            temp.pop(1) #get rid of the position
            for i in range(len(temp)): #remove extraneous characters
                for char in ['$', '>', ',']:
                    temp[i] = temp[i].replace(char,'')
            temp[1] = float(temp[1])
            for i in range(5,8):
                temp[i] = int(temp[i])
            
            #gets player_id from player_id_dict, except block handles case where players name is different among pages
            try:
                p_id = player_id_dict[temp[0]]
            except KeyError:
                if temp[0] in name_errors_dict:
                    p_id = player_id_dict[name_errors_dict[temp[0]]]
                if ' '.join(temp[0].split(' ')[:2]) in player_id_dict:
                    p_id = player_id_dict[' '.join(temp[0].split(' ')[:2])]
                else:
                    continue
                
            temp.insert(0, int(year))
            temp.insert(0, p_id)
            
            if temp[5]==temp[6]:
                change_team = 0
            else:
                change_team = 1
            temp.insert(7, change_team)    
            
            
            data.append(tuple(temp))
    
    return data

In [10]:
free_agents_data = get_free_agent_data(YEARS, player_id_dict)

This next cell will get the length of the free_agents_data array and an example tuple from this array.

In [11]:
print(len(free_agents_data))
print(free_agents_data[0])

360
(504, 2019, 'Klay Thompson', 29.4, 'UFA', 'GSW', 'GSW', 0, 5, 189903600, 37980720)


### Loading Data Into Database

Now that I have scraped the data we need from the web, I will load this data into a sqlite database. First, I will make sure, that I am starting with a clean slate by removing a past database if it exists and reloading it from the .sql file.

In [12]:
!rm data.db
!sqlite3 data.db < data.sql

Connecting to the database

In [13]:
con = sqlite3.connect('data.db')

Populating the players table

In [14]:
con.executemany('INSERT INTO players (id, name, position, height, weight, birthday, country, rookie_year, college) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)', player_data);
con.commit()

Checking to make sure the size of the table matches the size of the player_data array from earlier.

In [15]:
list(con.execute('SELECT COUNT(*) FROM players'))

[(756,)]

Populating the stats table

In [16]:
con.executemany('INSERT INTO stats (id, year, name, age, team, games, games_started, minutes, fg, fga, fg_per, three_fg, three_fga, three_fg_per, two_fg, two_fga, two_fg_per, efg_per, ft, fta, ft_per, orb, drb, trb, ast, stl, blk, tov, pfl, pts) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', stats_data);
con.commit()

Checking to make sure the size of the table matches the size of the stats_data array from earlier.

In [17]:
list(con.execute('SELECT COUNT(*) FROM stats'))

[(1594,)]

Populating the contracts table

In [18]:
con.executemany('INSERT INTO contracts (id, year, name, age, type, old_team, new_team, chg_team, length, total_dollars, avg_dollars) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', free_agents_data);
con.commit()

Checking to make sure the size of the table matches the size of the free_agents_data array from earlier.

In [19]:
list(con.execute('SELECT COUNT(*) FROM contracts'))

[(360,)]

In [20]:
con.close()